In [1]:
import pandas as pd

In [2]:
ruta_archivo =r"C:\Users\Andres Rodgers\OneDrive - Universidad Nacional Abierta y a Distancia\1. Especialización\Diagnostico APP\data\raw\Maestro_Final_SinJerarquia2.2.xlsx"

In [3]:
# Cargar el archivo, forzando las columnas de códigos como texto
df = pd.read_excel(ruta_archivo, dtype={
    "Cod. Prod. V14": str,
    "Cod. Prod. V26 ES": str
})

# Renombrar columnas clave
df = df.rename(columns={
    "Cod. Prod. V14": "codigo_v14",
    "Nombre Producto V4": "nombre_v14",
    "Cod. Prod. V26 ES": "codigo_v26",
    "Nombre Producto V26 ES": "nombre_v26",
    "plantilla_final": "plantilla"
})

# Limpiar códigos (evitar '.0' si vinieran así)
df["codigo_v14"] = df["codigo_v14"].str.strip().str.replace(".0", "", regex=False)
df["codigo_v26"] = df["codigo_v26"].str.strip().str.replace(".0", "", regex=False)

# Filtrar filas con características
df_filtrado = df[df["plantilla"].notna()].copy()

# Completar código y nombre final con prioridad V26
df_filtrado["codigo_final"] = df_filtrado["codigo_v26"].fillna(df_filtrado["codigo_v14"])
df_filtrado["nombre_final"] = df_filtrado["nombre_v26"].fillna(df_filtrado["nombre_v14"])

# Selección de columnas finales
df_filtrado = df_filtrado[["codigo_final", "nombre_final", "codigo_v14", "nombre_v14", "codigo_v26", "nombre_v26", "plantilla"]]
df_filtrado.head()

,codigo_final,nombre_final,codigo_v14,nombre_v14,codigo_v26,nombre_v26,plantilla
0,10151704,Semillas o plantulas de pasto,10151704,Semillas O Plantulas De Pasto,10151704,Semillas o plantulas de pasto,Nombre; Tipo
1,10161501,Olivos,10161501,Arboles Y Arbustos,10161501,Olivos,Nombre; Tipo; Longitud
2,10171504,Compost,10171504,Abono,10171504,Compost,Nombre; Tipo; Ingrediente activo; Aplicación; ...
3,10171605,"Mezclas de nitrogeno, fosforo y potasio NPK",10171605,Fertilizante,10171605,"Mezclas de nitrogeno, fosforo y potasio NPK",Características adicionales; Color; Ingredient...
4,10171701,Herbicida,10171701,Matamalezas,10171701,Herbicida,Aplicación; Aspecto físico; Nombre comercial; ...


In [4]:
# ✅ BLOQUE CORRECTO ÚNICO
df_codigos = df_filtrado[["codigo_final", "codigo_v14", "nombre_final"]].dropna(subset=["codigo_final", "nombre_final"])
df_codigos = df_codigos.rename(columns={
    "codigo_final": "code_v26",
    "codigo_v14": "code_v14",
    "nombre_final": "product_name"
})
df_codigos = df_codigos.drop_duplicates(subset=["code_v26"])

In [5]:
caracteristicas = []

# Iterar por cada fila con características
for _, row in df_filtrado.iterrows():
    codigo = row["codigo_final"]
    plantilla = row["plantilla"]
    
    claves = [clave.strip() for clave in plantilla.split(";") if clave.strip()]
    
    for jerarquia, clave in enumerate(claves, start=1):
        caracteristicas.append({
            "code_v26": codigo,
            "name": clave,
            "hierarchy_level": jerarquia
        })

# Convertir a DataFrame y eliminar duplicados por código + clave
df_characteristics = pd.DataFrame(caracteristicas).drop_duplicates(subset=["code_v26", "name"])
print(df_characteristics.head())


   code_v26      name  hierarchy_level
0  10151704    Nombre                1
1  10151704      Tipo                2
2  10161501    Nombre                1
3  10161501      Tipo                2
4  10161501  Longitud                3


In [6]:
# Crear tabla de mapeo V14 → V26
df_mappings = df_filtrado[
    df_filtrado["codigo_v14"].notna() & df_filtrado["codigo_v26"].notna()
][["codigo_v14", "codigo_v26"]].drop_duplicates()
df_mappings = df_mappings.rename(columns={
    "codigo_v14": "code_v14",
    "codigo_v26": "code_v26"
})

In [7]:
# 1. Total de filas
total_original = len(df)
total_filtradas = len(df_filtrado)

# 2. Códigos únicos
codigos_unicos = df_codigos["code_v26"].nunique()

# 3. Características por código
carac_por_codigo = df_characteristics.groupby("code_v26").size()
promedio_caracteristicas = carac_por_codigo.mean()

# 4. Verificar códigos o nombres faltantes
faltantes_codigo = df_filtrado["codigo_final"].isna().sum()
faltantes_nombre = df_filtrado["nombre_final"].isna().sum()

# 5. Duplicados de código + clave
duplicados = df_characteristics.duplicated(subset=["code_v26", "name"]).sum()

# Mostrar resumen
print("🔍 VALIDACIÓN GENERAL")
print(f"- Filas originales: {total_original}")
print(f"- Filas con características: {total_filtradas}")
print(f"- Códigos únicos encontrados: {codigos_unicos}")
print(f"- Promedio de características por código: {promedio_caracteristicas:.2f}")
print(f"- Códigos faltantes: {faltantes_codigo}")
print(f"- Nombres faltantes: {faltantes_nombre}")
print(f"- Claves duplicadas por código (debería ser 0): {duplicados}")


🔍 VALIDACIÓN GENERAL
- Filas originales: 1323
- Filas con características: 1323
- Códigos únicos encontrados: 630
- Promedio de características por código: 5.40
- Códigos faltantes: 693
- Nombres faltantes: 693
- Claves duplicadas por código (debería ser 0): 0


In [8]:
import os

try:
    ruta_salida = r"C:\Users\Andres Rodgers\OneDrive - Universidad Nacional Abierta y a Distancia\1. Especialización\Diagnostico APP\data\processed"

    os.makedirs(ruta_salida, exist_ok=True)

    df_codigos.to_csv(os.path.join(ruta_salida, "codigos_unspsc.csv"), index=False, encoding="utf-8")
    df_mappings.to_csv(os.path.join(ruta_salida, "mapeos_v14_v26.csv"), index=False, encoding="utf-8")
    df_characteristics.to_csv(os.path.join(ruta_salida, "caracteristicas.csv"), index=False, encoding="utf-8")

    print("✅ Archivos exportados correctamente:")
    print(f"- {ruta_salida}\\codigos_unspsc.csv")
    print(f"- {ruta_salida}\\mapeos_v14_v26.csv")
    print(f"- {ruta_salida}\\caracteristicas.csv")

except Exception as e:
    print("❌ Error al exportar los archivos CSV:")
    print(str(e))


✅ Archivos exportados correctamente:
- C:\Users\Andres Rodgers\OneDrive - Universidad Nacional Abierta y a Distancia\1. Especialización\Diagnostico APP\data\processed\codigos_unspsc.csv
- C:\Users\Andres Rodgers\OneDrive - Universidad Nacional Abierta y a Distancia\1. Especialización\Diagnostico APP\data\processed\mapeos_v14_v26.csv
- C:\Users\Andres Rodgers\OneDrive - Universidad Nacional Abierta y a Distancia\1. Especialización\Diagnostico APP\data\processed\caracteristicas.csv


In [9]:
df_codigos = df_codigos.drop_duplicates(subset=["code_v26"])
print(f"✅ Códigos únicos listos para insertar: {df_codigos['code_v26'].nunique()}")

✅ Códigos únicos listos para insertar: 630


# Carga archivos BDM

In [10]:
from sqlalchemy import create_engine, text

usuario = "root"
contraseña = "Sql150796*"
host = "127.0.0.1"
puerto = 3306
base_datos = "base_datos_maestra"

try:
    engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
        print("✅ Conexión exitosa con la base de datos.")
except Exception as e:
    print("❌ Error al conectar:")
    print(e)


✅ Conexión exitosa con la base de datos.


In [11]:
try:
    df_codigos.to_sql("unspsc_codes", con=engine, if_exists="append", index=False)
    print("✅ Datos insertados correctamente en unspsc_codes.")
except Exception as e:
    print("❌ Error al insertar en unspsc_codes:")
    print(e)

✅ Datos insertados correctamente en unspsc_codes.


In [12]:
duplicados = df_codigos[df_codigos.duplicated(subset=["code_v26"], keep=False)]
duplicados = duplicados.sort_values("code_v26")

print(f"🔁 Total códigos duplicados en df_codigos: {duplicados['code_v26'].nunique()}")
duplicados[["code_v26", "code_v14", "product_name"]].to_string(index=False)


🔁 Total códigos duplicados en df_codigos: 0


'Empty DataFrame\nColumns: [code_v26, code_v14, product_name]\nIndex: []'

In [13]:
# Obtener ids desde la tabla ya insertada
df_ids = pd.read_sql("SELECT id, code_v26 FROM unspsc_codes", con=engine)

# Hacer merge para obtener el ID
df_mappings = df_mappings.merge(df_ids, on="code_v26", how="inner")
df_mappings = df_mappings.rename(columns={"id": "unspsc_code_id"})

# Reorganizar columnas finales
df_mappings = df_mappings[["code_v14", "unspsc_code_id"]]

# Insertar en tabla code_mappings
try:
    df_mappings.to_sql("code_mappings", con=engine, if_exists="append", index=False)
    print("✅ Mapeos insertados en code_mappings.")
except Exception as e:
    print("❌ Error al insertar en code_mappings:")
    print(e)


✅ Mapeos insertados en code_mappings.


In [14]:
# Asociar id con características
df_characteristics = df_characteristics.merge(df_ids, left_on="code_v26", right_on="code_v26", how="inner")
df_characteristics = df_characteristics.rename(columns={"id": "unspsc_code_id"})

# Seleccionar columnas finales
df_characteristics = df_characteristics[["unspsc_code_id", "name", "hierarchy_level"]]

# Insertar en tabla characteristics
try:
    df_characteristics.to_sql("characteristics", con=engine, if_exists="append", index=False)
    print("✅ Características insertadas en characteristics.")
except Exception as e:
    print("❌ Error al insertar en characteristics:")
    print(e)


✅ Características insertadas en characteristics.
